In [27]:
from langchain.chains import APIChain
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders import FireCrawlLoader


In [79]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [42]:
import os

In [150]:
llm = ChatGroq(model='mixtral-8x7B-32768',api_key=os.getenv('GROQ_API_KEY'))
# llm = ChatOpenAI(model='gpt-3.5-turbo',api_key=os.getenv('OPENAI_API_KEY'))

In [33]:
loader = FireCrawlLoader(
    api_key=os.getenv('FIRECRAWL_API_KEY'), url="https://developer.nytimes.com/docs/books-product/overview", mode="scrape"
)

In [35]:
docs = loader.load()

In [40]:
api_docs = docs[0].page_content

In [107]:
chain = APIChain.from_llm_and_api_docs(llm=llm,api_docs=api_docs,limit_to_domains=None,verbose=True)

In [110]:
question = 'give me the list of books written by richard dawkins.Dont forget to include their publication date and summary.Your api key is {}'.format('9HeqmbZqQObfQsw0NHn66ppKB4enZOpi')

In [111]:
chain.invoke(question)



> Entering new APIChain chain...
https://api.nytimes.com/svc/books/v3/reviews.json?author=Richard+Dawkins&api-key=9HeqmbZqQObfQsw0NHn66ppKB4enZOpi
{"status":"OK","copyright":"Copyright (c) 2024 The New York Times Company.  All Rights Reserved.","num_results":7,"results":[{"url":"http:\/\/www.nytimes.com\/2003\/10\/05\/books\/books-in-brief-nonfiction-431303.html","publication_dt":"2003-10-05","byline":"CHRISTINE KENNEALLY","book_title":"Devil's Chaplain : Reflections on Hope, Lies, Science, and Love","book_author":"Richard Dawkins","summary":"","uuid":"00000000-0000-0000-0000-000000000000","uri":"nyt:\/\/book\/00000000-0000-0000-0000-000000000000","isbn13":["9780618335404"]},{"url":"http:\/\/www.nytimes.com\/2013\/09\/19\/books\/richard-dawkinss-appetite-for-wonder-a-memoir.html","publication_dt":"2013-09-19","byline":"JANET MASLIN","book_title":"An Appetite for Wonder","book_author":"Richard Dawkins","summary":"In \u201cAn Appetite for Wonder,\u201d the evolutionary biologist Richar

{'question': 'give me the list of books written by richard dawkins.Dont forget to include their publication date and summary.Your api key is 9HeqmbZqQObfQsw0NHn66ppKB4enZOpi',
 'output': 'The API response provides a list of 7 books written by Richard Dawkins along with their publication dates and summaries. The books include titles such as "Devil\'s Chaplain", "An Appetite for Wonder", "The God Delusion", "Blind Watchmaker", "Unweaving the Rainbow", "The Ancestor\'s Tale", and "Brief Candle in the Dark". Each book has a unique summary that gives a brief overview of its content.'}

In [151]:
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents.agent_types import AgentType
from langchain.agents import Tool, initialize_agent
from langchain_community.utilities import WikipediaAPIWrapper


In [152]:
wikipedia = WikipediaAPIWrapper()

wikipedia_tool = Tool(name="Wikipedia",func=wikipedia.run,description='''A useful tool for searching the Internet 
                        to find information on world events, issues, dates, years, etc. Worth 
                        using for general topics. Use precise questions.''')

In [153]:
problem_chain = LLMMathChain.from_llm(llm=llm)


In [154]:
math_tool = Tool.from_function(name="Calculator",
                              func=problem_chain.run,
                              description='''Useful for when you need to answer questions 
            about math. This tool is only for math questions and nothing else. Only input
            math expressions.''')

In [155]:
from langchain.prompts.prompt import PromptTemplate

In [156]:
word_problem_template = """You are a reasoning agent tasked with solving 
        the user's equation based questions. Logically arrive at the solution, and be 
        factual. In your answers, clearly detail the steps involved and give the 
        final answer. Provide the response in bullet points. 
        Question  {question} Answer"""

math_assistant_prompt = PromptTemplate(input_variables=["question"],
                                       template=word_problem_template
                                       )
word_problem_chain = LLMChain(llm=llm,
                              prompt=math_assistant_prompt)
word_problem_tool = Tool.from_function(name="Reasoning Tool",
                                       func=word_problem_chain.run,
                                       description='''Useful for when you need 
                                            to answer logic-based/reasoning questions.''')
                                                                                

In [157]:
agent = initialize_agent(
    tools=[wikipedia_tool, math_tool, word_problem_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
)

In [158]:
response = agent.invoke({"input": '''What is (x*x - y*y)/z, if x is 3 y is 4 and z is 13'''})

In [159]:
print(response)

{'input': 'What is (x*x - y*y)/z, if x is 3 y is 4 and z is 13', 'output': '-0.5384615384615384'}


In [160]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [166]:
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to spanish:"
    "\n\n{Text}"
)

In [167]:
first_chain = first_prompt | llm | StrOutputParser()

In [168]:
second_prompt = ChatPromptTemplate.from_template(
    "What language is the following text:\n\n{Spanish_Translation}"
)

In [169]:
second_chain = second_prompt | llm | StrOutputParser()

In [170]:
combined_chains = {"Spanish_Translation": first_chain} | RunnablePassthrough.assign(language=second_chain)

In [171]:
review = '''
Being in a start-up myself, I read this book to find comfort 
and confirmation about the volatility and emotional roller-coaster 
that comes with a working at the brink of software-as-a service. 
This book was just what it promised - interviews from some of the 
great successes in SV and elsewhere, their humble and sometimes 
difficult beginnings ('against all odds') that I am experiencing 
right now. It's not a literary piece of work - never intended to 
be, on the contrary, I felt the writing style was just what fits 
with a life in the fast lane - little time, easy to read chapters, 
inspiring and thank god, very 'down to earth.'

The one critical point I would like to make: I am somewhat 
perplexed how the companies were chosen - there are so many 
other companies that could have fit the profile which seem 
much more of a success than some of the companies/products in 
the book (gmail? Comm'on, I guess the author wanted to have 
Google in there, but didn't get an interview so she went with 
gmail?). Other great companies are easy to find - they don't
even need to be in the consumer space. How about Salesforce.com? 
I definitely liked the mix of 'new' and 'experienced' start ups. 

This book was a breeze to read and insightful for us start-up 
enterpreneurs.
'''

In [172]:
response = combined_chains.invoke({"Text": review})

In [175]:
print(response['language'])

The text is in Spanish. It is a review of a book about start-ups and the experiences of successful entrepreneurs. The reviewer is expressing their thoughts on the book, including their appreciation for the stories of successful founders and their criticism of the selection of companies featured in the book. They mention that the book was easy to read and provided valuable insights for start-up founders like themselves.


In [176]:
rock_template = """You are a very smart rock and roll professor. \
You are great at answering questions about rock and roll in a concise\
and easy to understand manner.

Here is a question:
{input}"""


politics_template = """You are a very good politics professor. \
You are great at answering politics questions..

Here is a question:
{input}"""


history_template = """You are a very good history teacher. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods.

Here is a question:
{input}"""


sports_template = """ You are a sports teacher.\
You are great at answering sports questions.

Here is a question:
{input}"""

In [177]:
prompt_infos = [
    {
        "name": "rock", 
        "description": "Good for answering questions about rock and roll", 
        "prompt_template": rock_template
    },
    {
        "name": "politics", 
        "description": "Good for answering questions about politics", 
        "prompt_template": politics_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "sports", 
        "description": "Good for answering questions about sports", 
        "prompt_template": sports_template
    }
]

In [178]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [180]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

In [184]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [185]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.

REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [186]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(
    llm=llm, 
    prompt=router_prompt
)

In [191]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [194]:
print(destination_chains.keys())

dict_keys(['rock', 'politics', 'History', 'sports'])


In [192]:
chain.run("Who was Joe DiMaggio? Respond in less than 100 words")



> Entering new MultiPromptChain chain...
History: {'input': 'Who was Joe DiMaggio?'}
> Finished chain.


"Joe DiMaggio was a renowned professional baseball player who played as a center fielder for the New York Yankees in Major League Baseball (MLB) from 1936 to 1951. He is considered one of the greatest baseball players of all time and was a key figure in the Yankees' success during his career.\n\nDiMaggio was born in Martinez, California, in 1914 to Italian immigrant parents. He grew up in a family of baseball players, with his older brothers Vince and Dominic also playing professionally. DiMaggio began his baseball career in the minor leagues before being signed by the Yankees in 1935.\n\nDiMaggio quickly established himself as one of the best players in the league, winning the American League (AL) Rookie of the Year award in 1936. He went on to win nine AL championships and three World Series titles with the Yankees, as well as three AL Most Valuable Player (MVP) awards.\n\nDiMaggio is perhaps best known for his 56-game hitting streak in 1941, which remains a Major League record. He w

In [195]:
chain.invoke('who won the last soccer world cup')



> Entering new MultiPromptChain chain...
sports: {'input': 'who won the last soccer world cup'}
> Finished chain.


{'input': 'who won the last soccer world cup',
 'text': 'The last soccer World Cup was won by France in 2018. They defeated Croatia 4-2 in the final match. If you have any other sports-related questions, feel free to ask!'}

In [196]:
chain.invoke(
    'What is (x*x - y*y)/z, if x is 3 y is 4 and z is 13'
)



> Entering new MultiPromptChain chain...
None: {'input': 'What is (x*x - y*y)/z, if x is 3 y is 4 and z is 13'}
> Finished chain.


{'input': 'What is (x*x - y*y)/z, if x is 3 y is 4 and z is 13',
 'text': "To find the value of the expression (x*x - y*y)/z when x is 3, y is 4, and z is 13, we need to substitute the given values into the expression:\n\n(x*x - y*y)/z = (3*3 - 4*4)/13\n\nFirst, let's calculate the values inside the parentheses:\n\n3*3 = 9\n4*4 = 16\n\nNow, substitute these values back into the expression:\n\n(9 - 16)/13\n\nNext, subtract the numbers inside the parentheses:\n\n-7/13\n\nThe expression (x*x - y*y)/z, with x equal to 3, y equal to 4, and z equal to 13, is equal to -7/13."}

In [197]:
chain.invoke('who in narendra modi')



> Entering new MultiPromptChain chain...
None: {'input': 'who is narendra modi'}
> Finished chain.


{'input': 'who is narendra modi',
 'text': "Narendra Modi is the current Prime Minister of India, serving in this position since May 2014. He is a member of the Bharatiya Janata Party (BJP) and was the Chief Minister of the state of Gujarat from 2001 to 2014. Modi was born on September 17, 1950, in Vadnagar, Gujarat, and he grew up in a modest family. He joined the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist organization, at a young age and rose through its ranks. Modi has a bachelor's degree in political science from Gujarat University.\n\nModi's tenure as Chief Minister of Gujarat was marked by significant economic growth and development in the state. He was credited with improving the state's infrastructure, attracting investment, and promoting entrepreneurship. However, his tenure was also marred by controversy over his handling of the 2002 Gujarat riots, which resulted in the deaths of over 1,000 people, mostly Muslims. Modi was accused of failing to prevent the violenc

In [198]:
chain.invoke('who is donald trump')



> Entering new MultiPromptChain chain...
politics: {'input': 'who is Donald Trump'}
> Finished chain.


{'input': 'who is Donald Trump',
 'text': 'Donald Trump is an American politician and businessman who served as the 45th president of the United States from January 2017 to January 2021. He was previously a real estate mogul and hosted the reality TV show "The Apprentice." Trump is a member of the Republican Party and is known for his controversial policies and statements, as well as his use of social media to communicate with the public. Prior to his presidency, he had never held political office. Trump was born and raised in Queens, New York, and received a degree in economics from the Wharton School of the University of Pennsylvania. He is a controversial figure and his presidency was marked by political polarization and division.'}